In [14]:
import cv2
import joblib
import time

color = (255, 0, 0)
cap = cv2.VideoCapture(0)
model = joblib.load("../weight/both eyes/best_random_forest_model.pkl")
class_names = ['Drowsy', 'Non Drowsy']

# Inisialisasi variabel FPS
fps_start_time = time.time()
fps_counter = 0

while (cap.isOpened()):
    ret , frame = cap.read()
    if ret:
        eyes_roi = []
        eyes = cv2.CascadeClassifier(r'github\utils\two_eyes_big.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        eyes_detect = eyes.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=3)

        for (x, y, w, h) in eyes_detect:
            face_roi = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x-5, y-5), (x + w+5, y + h+5), (0, 255, 0), 4)

            convert = face_roi/255.0
            mata = cv2.resize(convert, (149,37), interpolation = cv2.INTER_AREA)
            gambar_flat=[mata.flatten()]
            prediksi= model.predict(gambar_flat)
            cv2.putText(frame, f"{class_names[prediksi[0]]}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2)
        
        fps_counter += 1
        if (time.time() - fps_start_time) > 1:
            fps = fps_counter / (time.time() - fps_start_time)
            fps_counter = 0
            fps_start_time = time.time()

        cv2.putText(frame, f"FPS : {round(fps, 2)}", (frame.shape[1] - 200, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Video Original" , frame)
        
    else:
        print('no video')
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1